이탈 고객 예측


##### 데이터전처리(결측치처리, 클래스 불균형 확인 및 처리)
##### 특성공학 - 피처 선정 또는 피처생성
##### 베이스 모델선정
    - 분류모델 : LogisticRegression
    - 클래스 리포트, 과적합여부
##### 베이스 모델의 성능을 상회 하는 모델과 하이퍼 파라메터 조합을 찾아서 성능을 출력


In [2]:
import pandas as pd
url  = 'https://raw.githubusercontent.com/pia222sk20/python/refs/heads/main/data/tellecom.csv'
df = pd.read_csv(url)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


| 컬럼명                  | 설명                                                                         |
| :------------------- | :------------------------------------------------------------------------- |
| **customerID**       | 고객 고유 ID (식별용, 모델에서는 제외)                                                   |
| **gender**           | 성별 (‘Male’, ‘Female’)                                                      |
| **SeniorCitizen**    | 노년층 여부 (0=아님, 1=노년층)                                                       |
| **Partner**          | 배우자 유무 (‘Yes’, ‘No’)                                                       |
| **Dependents**       | 부양가족 여부 (‘Yes’, ‘No’)                                                      |
| **tenure**           | 가입 기간 (개월 단위)                                                              |
| **PhoneService**     | 전화 서비스 사용 여부 (‘Yes’, ‘No’)                                                 |
| **MultipleLines**    | 다중 회선 여부 (‘Yes’, ‘No’, ‘No phone service’)                                 |
| **InternetService**  | 인터넷 서비스 종류 (‘DSL’, ‘Fiber optic’, ‘No’)                                    |
| **OnlineSecurity**   | 온라인 보안 서비스 (‘Yes’, ‘No’, ‘No internet service’)                            |
| **OnlineBackup**     | 온라인 백업 서비스 (‘Yes’, ‘No’, ‘No internet service’)                            |
| **DeviceProtection** | 기기 보호 서비스 (‘Yes’, ‘No’, ‘No internet service’)                             |
| **TechSupport**      | 기술 지원 서비스 (‘Yes’, ‘No’, ‘No internet service’)                             |
| **StreamingTV**      | TV 스트리밍 서비스 사용 (‘Yes’, ‘No’, ‘No internet service’)                        |
| **StreamingMovies**  | 영화 스트리밍 서비스 (‘Yes’, ‘No’, ‘No internet service’)                           |
| **Contract**         | 계약 형태 (‘Month-to-month’, ‘One year’, ‘Two year’)                           |
| **PaperlessBilling** | 전자 청구서 여부 (‘Yes’, ‘No’)                                                    |
| **PaymentMethod**    | 납부 방식 (‘Electronic check’, ‘Mailed check’, ‘Bank transfer’, ‘Credit card’) |
| **MonthlyCharges**   | 월 요금                                                                       |
| **TotalCharges**     | 총 요금 (가입 기간 * 월 요금에 비례)                                                    |
| **Churn**            | 💡 **타깃 컬럼 (이탈 여부)** — ‘Yes’면 이탈, ‘No’면 유지                                 |


In [3]:
df.columns
# column명 확인

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [4]:
#결측치확인
df.isnull().sum() #전체요약
#결측치는 없음

#클래스 불균형
df['Churn'].value_counts(normalize=True)
#이탈안한 고객 73% 이탈한 고객 27% 불균형 데이터임

Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

In [5]:
%pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [7]:
# 결측치 타입확인
print(df.info())
print(df.isnull().sum())
#totalcharges가 object가 아닌데 object로 되어있음, 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
# 각 컬럼의 결측치 개수
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [24]:
# origin 열의 문자열 자료형을 범주형으로 변환
df['TotalCharges'] = df['TotalCharges'].astype('str')     
print(df['TotalCharges'].dtypes) 
#다시 확인
df.info()

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   o

In [10]:
#결측치 대체
df["TotalCharges"] = df["TotalCharges"].fillna(df["MonthlyCharges"] * df["tenure"])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np

# 데이터 로드
url = "https://raw.githubusercontent.com/pia222sk20/python/refs/heads/main/data/tellecom.csv"
df = pd.read_csv(url)

# y = Churn, X = 나머지
target = "Churn"

# 불필요한 컬럼 제거
drop_cols = ["customerID"]
num_cols = ["tenure", "MonthlyCharges", "TotalCharges"]
cat_cols = [
    "gender","SeniorCitizen","Partner","Dependents","PhoneService","MultipleLines",
    "InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport",
    "StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod"]


In [14]:
X = df.drop(columns=drop_cols + [target]).copy()
y = df[target].copy()

X["TotalCharges"] = pd.to_numeric(X["TotalCharges"].astype(str), errors="coerce")
X["TotalCharges"] = X["TotalCharges"].fillna(X["MonthlyCharges"] * X["tenure"])

X_num = X[num_cols].copy()
X_cat = X[cat_cols].copy()

X_cat = X_cat.astype("string").fillna("Unknown")
X_cat_ohe = pd.get_dummies(X_cat, drop_first=False, dtype=np.float64)

scaler = StandardScaler()
X_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_num),
    columns=X_num.columns,
    index=X_num.index
)

#합산
X1 = pd.concat([X_num_scaled, X_cat_ohe], axis=1)

# 7) 학습/검증 분리
X_train, X_test, y_train, y_test = train_test_split(
    X1, y, test_size=0.2, random_state=42, stratify=y
)

print("X_ready shape:", X1.shape)
print("Train/Test:", X_train.shape, X_test.shape)
print("y 분포(Train):")
print(y_train.value_counts(normalize=True).round(3))

X_ready shape: (7043, 46)
Train/Test: (5634, 46) (1409, 46)
y 분포(Train):
Churn
No     0.735
Yes    0.265
Name: proportion, dtype: float64


In [15]:
# 피처 생성 단계
X = X.copy()

# 1. 월평균 요금
X["AvgChargePerMonth"] = X["TotalCharges"] / X["tenure"].replace(0, np.nan)

# 2. 여러 서비스 사용 개수
service_cols = ["OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
X["ServiceCount"] = (X[service_cols] == "Yes").sum(axis=1)

# 3. 인터넷 가입 여부
X["HasInternet"] = (X["InternetService"] != "No").astype(int)

# 확인
print(X[["AvgChargePerMonth","ServiceCount","HasInternet"]].head())


   AvgChargePerMonth  ServiceCount  HasInternet
0          29.850000             1            1
1          55.573529             2            1
2          54.075000             2            1
3          40.905556             3            1
4          75.825000             0            1


In [16]:
# 수치형 변수만 뽑기
num_cols = ["tenure","MonthlyCharges","TotalCharges","AvgChargePerMonth","ServiceCount"]

# 상관관계 확인
corr = X[num_cols].corr()
print(corr)


                     tenure  MonthlyCharges  TotalCharges  AvgChargePerMonth  \
tenure             1.000000        0.247900      0.826178           0.246198   
MonthlyCharges     0.247900        1.000000      0.651174           0.996237   
TotalCharges       0.826178        0.651174      1.000000           0.650915   
AvgChargePerMonth  0.246198        0.996237      0.650915           1.000000   
ServiceCount       0.494263        0.724706      0.744827           0.722902   

                   ServiceCount  
tenure                 0.494263  
MonthlyCharges         0.724706  
TotalCharges           0.744827  
AvgChargePerMonth      0.722902  
ServiceCount           1.000000  
